In [ ]:
import pandas as pd
df = pd.read_csv("cleaned_data_combined_modified.csv")

In [ ]:
# df.describe()
df

id  \
0     716549   
1     715742   
2     727333   
3     606874   
4     505318   
...      ...   
1639  413395   
1640  386750   
1641  617178   
1642  630128   
1643  628736   

      Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)  \
0                                                     3                                                                       
1                                                     4                                                                       
2                                                     3                                                                       
3                                                     4                                                                       
4                                                     2                                                                       
...                                                 ...                                                                       
1639                                                  2                                                                       
1640                                                  3                                                                       
1641                                                  2                                                                       
1642                                                  1                                                                       
1643                                                  4                                                                       

     Q2: How many ingredients would you expect this food item to contain?  \
0                                                     6                     
1                                           bread, meet                     
2                                                     5                     
3                                                   6-7                     
4                                             3 or more                     
...                                                 ...                     
1639                                                3-5                     
1640                                             4 to 5                     
1641  It has a minimum of two ingredients, and on av...                     
1642                          3 or a finite amount more                     
1643  This food item typically contains 3 to 5 ingre...                     

     Q3: In what setting would you expect this food to be served? Please check all that apply  \
0            Week day lunch,At a party,Late night snack                                         
1            Week day lunch,At a party,Late night snack                                         
2     Week day lunch,Week day dinner,Weekend lunch,W...                                         
3     Week day lunch,Week day dinner,Weekend lunch,W...                                         
4     Week day lunch,Week day dinner,Weekend lunch,W...                                         
...                                                 ...                                         
1639                     Week day dinner,Weekend dinner                                         
1640                       Week day lunch,Weekend lunch                                         
1641            Week day lunch,Weekend lunch,At a party                                         
1642      Week day lunch,Weekend lunch,Late night snack                                         
1643  Week day lunch,Week day dinner,Weekend lunch,W...                                         

     Q4: How much would you expect to pay for one serving of this food item?  \
0                                                     5                        
1                                  5$ for 

For Q2

In [ ]:
import re
import math

def process_q2(text):
    if pd.isnull(text):
        return None

    # Try extracting numbers first
    numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
    numbers = [float(num) for num in numbers]

    # Filter numbers between 1 and 10 inclusive
    filtered_numbers = [num for num in numbers if 1 <= num <= 10]

    if filtered_numbers:
        result = sum(filtered_numbers) / len(filtered_numbers)
    elif ',' in text:
        return text.count(',') + 1
    else:
        result = len(text) / 5

    return math.ceil(result)

# Read your CSV file
df = pd.read_csv("cleaned_data_combined_modified.csv")

# Rename the column for readability
df = df.rename(columns={'Q2: How many ingredients would you expect this food item to contain?': 'Q2'})

# Apply the processing function to the Q2 column
df['Q2'] = df['Q2'].apply(process_q2)

# Save the updated DataFrame to a new CSV file
df.to_csv("updated_data.csv", index=False)

print("Column renamed, values processed with rounding up (and comma logic), and saved to updated_data.csv.")

Column renamed, values processed with rounding up (and comma logic), and saved to updated_data.csv.


Clean Up For Q3

In [ ]:
# Clean the column (replace 'Q3' with the actual column name)
column_name = "Q3: In what setting would you expect this food to be served? Please check all that apply"  # Update with actual column name
df[column_name] = df[column_name].str.lower().str.strip()  # Convert to lowercase and remove spaces
# print(df[column_name].head(10))

# Split responses into lists
df[column_name] = df[column_name].str.split(",")
# print(df[column_name].head(10))
# Define all possible categories
categories = ["week day lunch", "week day dinner", "weekend lunch", "weekend dinner",
              "at a party", "late night snack"]

# Convert to one-hot encoding
for category in categories:
    df[category] = df[column_name].apply(lambda x: 1 if category in x else 0)

# Drop the original column
df = df.drop(columns=[column_name])

# Display first few rows
# print(df.head())
df.to_csv("updated_data.csv", index=False)

df


Cleaned dataset saved to updated_data.csv


For Q4 and Q6


In [ ]:
import numpy as np
import re

def extract_numeric(value):
    if isinstance(value, str):
        value = value.lower().strip()

        # Check for numbers in the string
        numbers = re.findall(r'\d+', value)
        if numbers:
            return int(numbers[0])  # Take the first number found

        return "unknown"  # If no numbers are found, set as unknown
    return value  # If already numeric, keep it

def clean_drink(value):
    if isinstance(value, str):
        value = value.lower().strip()

        # Define mapping categories
        drink_map = {
            "coke": ["coke", "cola", "diet coke", "pepsi", "diet pepsi", "soda", "pop", "soft", "fanta", "sprite", "ginger"],
            "juice": ["juice", "nestea", "lemonade"],
            "tea": ["tea", "matcha"],
            "milk": ["ayran", "milk", "dairy"],
            "beer": ["beer", "wine", "alcoholic", "sake", "soju"],
            "water": ["water"],
            "soup": ["miso", "soup"]
        }
        value = re.sub(r'[^a-z0-9 ]', '', value).strip()
        # Match drink to category
        for category, keywords in drink_map.items():
            if any(keyword in value for keyword in keywords):
                return category
        return "unknown"  # If not recognized
    return "unknown"

def clean_csv(file_path, output_path):
    # Load the dataset
    df = pd.read_csv(file_path, dtype=str)  # Load everything as string

    # Convert column names to a standardized format (lowercase, no spaces)
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    # Remove duplicate rows
    df = df.drop_duplicates()

    # Handle missing values by filling with 'unknown'
    df = df.fillna("unknown")


    # Process Q4 (extract first numeric value or set as 'unknown')
    if 'q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?' in df.columns:
        df['q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?'] = df[
            'q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?'
        ].apply(lambda x: extract_numeric(x) if isinstance(x, str) else "unknown")

    # Process Q6 (normalize drink names into categories)
    if 'q6:_what_drink_would_you_pair_with_this_food_item?' in df.columns:
        df['q6:_what_drink_would_you_pair_with_this_food_item?'] = df[
            'q6:_what_drink_would_you_pair_with_this_food_item?'
        ].apply(clean_drink)

    # Save cleaned dataset
    df.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")

# Example usage
clean_csv("updated_data.csv", "updated_data.csv")
df = pd.read_csv("updated_data.csv", dtype=str)

df.to_csv("updated_data.csv", index=False)
df


Cleaned dataset saved to updated_data.csv


id  \
0     716549   
1     715742   
2     727333   
3     606874   
4     505318   
...      ...   
1639  413395   
1640  386750   
1641  617178   
1642  630128   
1643  628736   

     q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)  \
0                                                     3                                                                      
1                                                     4                                                                      
2                                                     3                                                                      
3                                                     4                                                                      
4                                                     2                                                                      
...                                                 ...                                                                      
1639                                                  2                                                                      
1640                                                  3                                                                      
1641                                                  2                                                                      
1642                                                  1                                                                      
1643                                                  4                                                                      

     q2  \
0     6   
1     2   
2     5   
3     7   
4     3   
...  ..   
1639  4   
1640  5   
1641  2   
1642  3   
1643  4   

     q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?  \
0                                                     5                        
1                                                     5                        
2                                                    10                        
3                                                     3                        
4                                                     5                        
...                                                 ...                        
1639                                                  5                        
1640                                                 12                        
1641                                                 10                        
1642                                                  1                        
1643                                                100                        

     q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?  \
0                     Cloudy with a Chance of Meatballs                
1                 All sort of american young boy movies                
2                                          action movie                
3                                             Mamma Mia                
4                     Cloudy with a chance of meatballs                
...                                                 ...                
1639                                    Romantic Movies                
1640                                One Piece Film Gold                
1641                     I think of the anime Food Wars                
1642                                        John Wick 3                
1643  I think of Cars 2 when thinking of this food i...                

     q6:_what_drink_would_you_pair_with_this_food_item?  \
0                                                  coke   
1                                                  coke   
2                                                  coke   
3                                                  coke   
4                                                 

Clean Up For Q7

In [ ]:
column_name = "q7:_when_you_think_about_this_food_item,_who_does_it_remind_you_of?"  # Update with actual column name
df[column_name] = df[column_name].str.lower().str.strip()  # Convert to lowercase and remove spaces
# print(df[column_name].head(10))

# Split responses into lists
df[column_name] = df[column_name].apply(lambda x: x.split(",") if isinstance(x, str) else [])
print(df[column_name].head(10))
# Define all possible categories
categories = ["parents","siblings","friends","teachers","strangers"]

# Convert to one-hot encoding
for category in categories:
    df[category] = df[column_name].apply(lambda x: 1 if category in x else 0)

# Drop the original column
df = df.drop(columns=[column_name])

# Display first few rows
# print(df.head())
df.to_csv("updated_data.csv", index=False)

df


0                         [friends]
1    [friends, teachers, strangers]
2                         [friends]
3     [siblings, friends, teachers]
4               [siblings, friends]
5               [siblings, friends]
6                         [friends]
7                         [friends]
8                         [unknown]
9      [parents, siblings, friends]
Name: q7:_when_you_think_about_this_food_item,_who_does_it_remind_you_of?, dtype: object


id  \
0     716549   
1     715742   
2     727333   
3     606874   
4     505318   
...      ...   
1639  413395   
1640  386750   
1641  617178   
1642  630128   
1643  628736   

     q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)  \
0                                                     3                                                                      
1                                                     4                                                                      
2                                                     3                                                                      
3                                                     4                                                                      
4                                                     2                                                                      
...                                                 ...                                                                      
1639                                                  2                                                                      
1640                                                  3                                                                      
1641                                                  2                                                                      
1642                                                  1                                                                      
1643                                                  4                                                                      

     q2  \
0     6   
1     2   
2     5   
3     7   
4     3   
...  ..   
1639  4   
1640  5   
1641  2   
1642  3   
1643  4   

     q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?  \
0                                                     5                        
1                                                     5                        
2                                                    10                        
3                                                     3                        
4                                                     5                        
...                                                 ...                        
1639                                                  5                        
1640                                                 12                        
1641                                                 10                        
1642                                                  1                        
1643                                                100                        

     q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?  \
0                     Cloudy with a Chance of Meatballs                
1                 All sort of american young boy movies                
2                                          action movie                
3                                             Mamma Mia                
4                     Cloudy with a chance of meatballs                
...                                                 ...                
1639                                    Romantic Movies                
1640                                One Piece Film Gold                
1641                     I think of the anime Food Wars                
1642                                        John Wick 3                
1643  I think of Cars 2 when thinking of this food i...                

     q6:_what_drink_would_you_pair_with_this_food_item?  \
0                                                  coke   
1                                                  coke   
2                                                  coke   
3                                                  coke   
4                                                 

Clean Up For Q8

In [ ]:
import re  # Regular expressions for extracting text inside parentheses

# Load the dataset
# df = pd.read_csv("cleaned_data_combined_modified.csv")

# Define the column name
column_name = "q8:_how_much_hot_sauce_would_you_add_to_this_food_item?"

# Ensure the column is a string and handle NaN values
df[column_name] = df[column_name].fillna("").astype(str)

# Extract text inside parentheses, but handle cases where there's no match
df[column_name] = df[column_name].apply(lambda x: re.search(r"\((.*?)\)", x).group(1) if re.search(r"\((.*?)\)", x) else None)

# Define spice level categories
categories = ["none", "mild", "medium", "hot"]

# One-hot encode the spice levels
for category in categories:
    df[category] = df[column_name].apply(lambda x: 1 if x == category else 0)

# Calculate average for each category (ignoring NaN values)
avg_values = df[categories].mean().fillna(0)

# Define a threshold for assigning 1 or 0
threshold = 0.4  # Adjust as needed

# Fill missing values based on the threshold
for category in categories:
    df.loc[df[column_name].isna(), category] = 1 if avg_values[category] > threshold else 0

# Drop the original Q8 column
df = df.drop(columns=[column_name])
df.to_csv("updated_data.csv", index=False)

# Display first few rows
df

id  \
0     716549   
1     715742   
2     727333   
3     606874   
4     505318   
...      ...   
1639  413395   
1640  386750   
1641  617178   
1642  630128   
1643  628736   

     q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)  \
0                                                     3                                                                      
1                                                     4                                                                      
2                                                     3                                                                      
3                                                     4                                                                      
4                                                     2                                                                      
...                                                 ...                                                                      
1639                                                  2                                                                      
1640                                                  3                                                                      
1641                                                  2                                                                      
1642                                                  1                                                                      
1643                                                  4                                                                      

     q2  \
0     6   
1     2   
2     5   
3     7   
4     3   
...  ..   
1639  4   
1640  5   
1641  2   
1642  3   
1643  4   

     q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?  \
0                                                     5                        
1                                                     5                        
2                                                    10                        
3                                                     3                        
4                                                     5                        
...                                                 ...                        
1639                                                  5                        
1640                                                 12                        
1641                                                 10                        
1642                                                  1                        
1643                                                100                        

     q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?  \
0                     Cloudy with a Chance of Meatballs                
1                 All sort of american young boy movies                
2                                          action movie                
3                                             Mamma Mia                
4                     Cloudy with a chance of meatballs                
...                                                 ...                
1639                                    Romantic Movies                
1640                                One Piece Film Gold                
1641                     I think of the anime Food Wars                
1642                                        John Wick 3                
1643  I think of Cars 2 when thinking of this food i...                

     q6:_what_drink_would_you_pair_with_this_food_item?  label week_day_lunch  \
0                                                  coke  Pizza              1   
1                                                  coke  Pizza              1   
2                                                  coke  Pizza              1   
3                    

MAke q6 a one hot

In [ ]:
# Cleaning part 6 into hot encodings and then saving it into csv
import pandas as pd
import numpy as np
import re

# Load your CSV
df = pd.read_csv("updated_data_with_vectors.csv")

# ---- One-hot encode Q6 ----
drink_map = {
    "coke": ["coke", "cola", "diet coke", "pepsi", "diet pepsi", "soda", "pop", "soft", "fanta", "sprite", "ginger"],
    "juice": ["juice", "nestea", "lemonade"],
    "tea": ["tea", "matcha"],
    "milk": ["ayran", "milk", "dairy"],
    "beer": ["beer", "wine", "alcoholic", "sake", "soju"],
    "water": ["water"],
    "soup": ["miso", "soup"]
}

# Find the q6 column
q6_col = [col for col in df.columns if "q6:" in col.lower()][0]

# Lowercase and fill missing
drink_responses = df[q6_col].fillna("").str.lower()

# Encode drink categories
for cat, keywords in drink_map.items():
    pattern = '|'.join(map(re.escape, keywords))
    df[f'drink_{cat}'] = drink_responses.str.contains(pattern).astype(int)

# Drop the original q6 column

# ---- One-hot encode label column using pd.get_dummies ----
label_onehot = pd.get_dummies(df["label"], prefix="label").astype(int)
df = df.drop(columns=[q6_col, "id"])
# Turn it into a numpy array
label_array = label_onehot.to_numpy()

# If you want to concatenate label back to df:
# df = pd.concat([df, label_onehot], axis=1)

# Preview
print("Label columns:", label_onehot.columns.tolist())
print("One-hot encoded label array:\n", label_array[500:1000])

df
print(df.columns)
df.to_csv("updated_data.csv", index=False)

# [pizza, shawarma, sushi] for the truth embedding

In [ ]:
# import pandas as pd
# import numpy as np
# import re

# def clean_csv(file_path, output_path):
#     # Load the dataset
#     df = pd.read_csv("cleaned_data_combined_modified.csv", dtype=str)  # Load everything as string to avoid automatic type inference

#     # Convert column names to a standardized format (lowercase, no spaces)
#     df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

#     # Remove duplicate rows
#     df = df.drop_duplicates()

#     # Handle missing values by filling with 'unknown'
#     df = df.fillna("unknown")

#     # Normalize text fields (strip leading/trailing spaces, convert to lowercase)
#     df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

#     # Save cleaned dataset
#     df.to_csv(output_path, index=False)
#     print(f"Cleaned dataset saved to {output_path}")

# # Example usage
# clean_csv("cleaned_data_combined_modified.csv", "cleaned_output.csv")
# df = pd.read_csv("cleaned_output.csv")
# df

Cleaned dataset saved to cleaned_output.csv


<ipython-input-69-4ecaef0e7444>:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)


id  \
0     716549   
1     715742   
2     727333   
3     606874   
4     505318   
...      ...   
1639  413395   
1640  386750   
1641  617178   
1642  630128   
1643  628736   

      q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)  \
0                                                     3                                                                       
1                                                     4                                                                       
2                                                     3                                                                       
3                                                     4                                                                       
4                                                     2                                                                       
...                                                 ...                                                                       
1639                                                  2                                                                       
1640                                                  3                                                                       
1641                                                  2                                                                       
1642                                                  1                                                                       
1643                                                  4                                                                       

     q2:_how_many_ingredients_would_you_expect_this_food_item_to_contain?  \
0                                                     6                     
1                                           bread, meet                     
2                                                     5                     
3                                                   6-7                     
4                                             3 or more                     
...                                                 ...                     
1639                                                3-5                     
1640                                             4 to 5                     
1641  it has a minimum of two ingredients, and on av...                     
1642                          3 or a finite amount more                     
1643  this food item typically contains 3 to 5 ingre...                     

     q3:_in_what_setting_would_you_expect_this_food_to_be_served?_please_check_all_that_apply  \
0            week day lunch,at a party,late night snack                                         
1            week day lunch,at a party,late night snack                                         
2     week day lunch,week day dinner,weekend lunch,w...                                         
3     week day lunch,week day dinner,weekend lunch,w...                                         
4     week day lunch,week day dinner,weekend lunch,w...                                         
...                                                 ...                                         
1639                     week day dinner,weekend dinner                                         
1640                       week day lunch,weekend lunch                                         
1641            week day lunch,weekend lunch,at a party                                         
1642      week day lunch,weekend lunch,late night snack                                         
1643  week day lunch,week day dinner,weekend lunch,w...                                         

     q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?  \
0                                                     5                        
1                                  5$ for 

In [ ]:
# import pandas as pd
# import re
# import math

# def process_q2(text):
#     if pd.isnull(text):
#         return None

#     # Otherwise, try to extract numbers
#     numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
#     numbers = [float(num) for num in numbers]

#     # Filter numbers between 1 and 10 inclusive
#     filtered_numbers = [num for num in numbers if 1 <= num <= 10]

#     if filtered_numbers:
#         result = sum(filtered_numbers) / len(filtered_numbers)
#     elif ',' in text:
#         return text.count(',') + 1
#     else:
#         result = len(text) / 5

#     return math.ceil(result)

# # Read your CSV file
# df = pd.read_csv("cleaned_data_combined_modified.csv")

# # Rename the column for readability
# df = df.rename(columns={'Q2: How many ingredients would you expect this food item to contain?': 'Q2'})

# # Apply the processing function to the Q2 column
# df['Q2'] = df['Q2'].apply(process_q2)

# # Save the updated DataFrame to a new CSV file
# df.to_csv("updated_data.csv", index=False)

# print("Column renamed, values processed with rounding up (and comma logic), and saved to updated_data.csv.")


Column renamed, values processed with rounding up (and comma logic), and saved to updated_data.csv.


embedding for Q5    

In [ ]:
import pandas as pd

def clean_text(text):
    if isinstance(text, str):
        return ' '.join(text.lower().strip().split())
    return text

def clean_csv(file_path, output_path):
    df = pd.read_csv(file_path)
    column_name = "q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?"  # Assuming first column is the target
    df[column_name] = df[column_name].apply(clean_text)

    # One-hot encoding for each unique movie
    unique_movies = df[column_name].unique()
    for movie in unique_movies:
        if len(movie) < 50:
          df[movie] = df[column_name].apply(lambda x: 1 if x == movie else 0)

    # Drop the original Q8 column
    df = df.drop(columns=[column_name])
    df.to_csv(output_path, index=False)

# Example usage:
clean_csv("updated_data.csv", "updated_data.csv")



<ipython-input-9-444d7f87f93f>:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[movie] = df[column_name].apply(lambda x: 1 if x == movie else 0)
<ipython-input-9-444d7f87f93f>:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[movie] = df[column_name].apply(lambda x: 1 if x == movie else 0)
<ipython-input-9-444d7f87f93f>:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts here is where it starts

In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("cleaned_data_combined_modified.csv")

def process_q2(text):
    if pd.isnull(text):
        return None
    numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
    numbers = [float(num) for num in numbers]
    filtered_numbers = [num for num in numbers if 1 <= num <= 10]
    if filtered_numbers:
        result = sum(filtered_numbers) / len(filtered_numbers)
    elif ',' in text:
        return text.count(',') + 1
    else:
        result = len(text) / 5
    return np.ceil(result)

def extract_numeric(value):
    if isinstance(value, str):
        value = value.lower().strip()
        numbers = re.findall(r'\d+', value)
        if numbers:
            return int(numbers[0])
        return "unknown"
    return value

def clean_drink(value):
    drink_map = {
        "coke": ["coke", "cola", "diet coke", "pepsi", "diet pepsi", "soda", "pop", "soft", "fanta", "sprite", "ginger"],
        "juice": ["juice", "nestea", "lemonade"],
        "tea": ["tea", "matcha"],
        "milk": ["ayran", "milk", "dairy"],
        "beer": ["beer", "wine", "alcoholic", "sake", "soju"],
        "water": ["water"],
        "soup": ["miso", "soup"]
    }
    if isinstance(value, str):
        value = re.sub(r'[^a-z0-9 ]', '', value.lower().strip())
        for category, keywords in drink_map.items():
            if any(keyword in value for keyword in keywords):
                return category
    return "unknown"

def one_hot_encode(df, column_name, categories):
    """
    Converts a column with comma-separated values into one-hot encoding.
    """
    if column_name in df.columns:
        df[column_name] = df[column_name].str.lower().str.strip()
        df[column_name] = df[column_name].apply(lambda x: x.split(",") if isinstance(x, str) else [])

        for category in categories:
            df[f"{column_name}_{category}"] = df[column_name].apply(lambda x: 1 if category in x else 0)

        df.drop(columns=[column_name], inplace=True)

    return df

def clean_dataset(file_path, output_path):
    df = pd.read_csv(file_path, dtype=str)
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    df = df.drop_duplicates().fillna("unknown")

    # Q2: Process numeric values
    q2_col = "q2:_how_many_ingredients_would_you_expect_this_food_item_to_contain?"
    if q2_col in df.columns:
        df['q2'] = df[q2_col].apply(process_q2)

    # Q4: Extract numbers
    q4_col = "q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?"
    if q4_col in df.columns:
        df['q4'] = df[q4_col].apply(extract_numeric)

    # Q6: Drink preference categorization
    q6_col = "q6:_what_drink_would_you_pair_with_this_food_item?"
    if q6_col in df.columns:
        df['q6'] = df[q6_col].apply(clean_drink)
        for cat in ["coke", "juice", "tea", "milk", "beer", "water", "soup"]:
            df[f'drink_{cat}'] = df['q6'].apply(lambda x: 1 if x == cat else 0)
        df.drop(columns=['q6'], inplace=True)

    # Q8: Hot sauce preference
    q8_col = "q8:_how_much_hot_sauce_would_you_add_to_this_food_item?"
    if q8_col in df.columns:
        df['q8'] = df[q8_col].apply(lambda x: re.search(r"\((.*?)\)", x).group(1) if re.search(r"\((.*?)\)", x) else None)
        for category in ["none", "mild", "medium", "hot"]:
            df[f'hot_sauce_{category}'] = df['q8'].apply(lambda x: 1 if x == category else 0)
        df.drop(columns=['q8'], inplace=True)

    # Q7: One-hot encoding for "Who does this food remind you of?"
    q7_col = "q7:_when_you_think_about_this_food_item,_who_does_it_remind_you_of?"
    q7_categories = ["parents", "siblings", "friends", "teachers", "strangers"]
    df = one_hot_encode(df, q7_col, q7_categories)

    # Q3: One-hot encoding for "Where would you expect this food to be served?"
    q3_col = "q3:_in_what_setting_would_you_expect_this_food_to_be_served?_please_check_all_that_apply"
    q3_categories = ["week day lunch", "week day dinner", "weekend lunch", "weekend dinner",
                     "at a party", "late night snack"]
    df = one_hot_encode(df, q3_col, q3_categories)

    # Save the cleaned dataset
    df.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")

# Example usage:
clean_dataset("cleaned_data_combined_modified.csv", "updated_data.csv")


Cleaned dataset saved to updated_data.csv


For Q5

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def clean_and_vectorize(file_path, output_path):
    """
    Reads a CSV file, processes the movie-related text column into a bag of words,
    replaces all non-numeric values in other columns with 0, and saves the cleaned DataFrame.

    :param file_path: Path to the input CSV file
    :param output_path: Path to save the cleaned CSV file
    """
    # Load the dataset
    df = pd.read_csv(file_path, dtype=str)  # Read as strings to ensure consistency

    # Define stop words
    stop_words = {
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
        "he", "him", "his", "she", "her", "hers", "it", "its", "they", "them", "their", "what", "which",
        "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been",
        "being", "have", "has", "had", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if",
        "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "between",
        "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
        "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there",
        "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "some",
        "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can",
        "will", "just", "don", "should", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren",
        "couldn", "didn", "doesn", "hadn", "hasn", "haven", "isn", "ma", "mightn", "mustn", "needn",
        "shan", "shouldn", "wasn", "weren", "won", "wouldn"
    }

    # Function to clean text and return list of words
    def text_to_words(text):
        if isinstance(text, str):
            words = text.lower().strip().split()
            return [word for word in words if word not in stop_words]
        return []

    # Process the movie column into a bag of words
    text_column = "q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?"
    word_counts = Counter()

    if text_column in df.columns:
        df[text_column] = df[text_column].apply(text_to_words)

        # Collect all unique words for bag-of-words representation
        for words in df[text_column]:
            word_counts.update(words)

        # Create bag-of-words DataFrame
        unique_words = sorted(word_counts.keys())  # Sort for consistency
        bow_df = pd.DataFrame(0, index=df.index, columns=unique_words)

        # Fill bag-of-words matrix
        for idx, words in enumerate(df[text_column]):
            for word in words:
                bow_df.at[idx, word] = 1  # Binary representation (1 if word appears)

        # Drop original column
        df.drop(columns=[text_column], inplace=True)

        # Merge the bag-of-words dataframe with the original dataframe
        df = pd.concat([df, bow_df], axis=1)

    # Function to check if a value is numeric
    def is_number(val):
        try:
            float(val)
            return True
        except ValueError:
            return False

    # # Replace non-numeric values in other columns with 0
    # df = df.applymap(lambda x: x if is_number(x) else 0)

    # Save the cleaned dataset
    df.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")

# Example usage:
clean_and_vectorize("updated_data.csv", "updated_data.csv")


Cleaned dataset saved to updated_data.csv


<ipython-input-87-e1fe063381a2>:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("updated_data.csv")


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
data = pd.read_csv("updated_data.csv")

# Replace non-numeric values with 0, except for the "label" column
for column in data.columns:
    if column != "label":
        data[column] = pd.to_numeric(data[column], errors='coerce').fillna(0)

data.to_csv("updated_data.csv", index=False)  # Saves without the index column


<ipython-input-3-032f080b0a70>:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("updated_data.csv")


cut id

In [ ]:
df = pd.read_csv("updated_data.csv")
df = df.drop("id", axis=1)
df.to_csv("updated_data.csv", index=False)


In [ ]:
df = pd.read_csv("updated_data.csv")
df = df.drop(df.columns[[1, 2, 3, 4]], axis=1)
df.to_csv("updated_data.csv", index=False)


In [ ]:
cut Q5 Optional

chop features


In [ ]:
import pandas as pd

# === Load your dataset with correct labels ===
df = pd.read_csv("updated_data.csv")  # Update path if needed

# === List of top features to keep ===
top_features = [
    'drink_coke', 'avengers', 'drink_tea', 'q4', 'hot_sauce_medium', 'ninja',
    'home', 'drink_water', 'teenage', 'mutant', 'turtles', 'alone', 'q2',
    'drink_beer', 'hot_sauce_hot', 'sushi',
    'q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)',
    'jiro', 'dreams', 'drink_juice', 'away', 'spirited', 'chance', 'drink_soup',
    'cloudy', 'meatballs', 'spiderman', 'nemo', 'pizza', 'finding', 'ratatouille',
    'spider-man', 'dictator', 'aladdin', 'tokyo', 'monsters', 'name', 'kill',
    'japanese', 'bill', 'drift', 'hot_sauce_mild', 'label'  # make sure to keep label
]

# === Filter dataset ===
df_reduced = df[top_features]

# === Save the reduced dataset ===
df_reduced.to_csv("updated_data.csv", index=False)

# Optional: print preview
print(df_reduced.head())

   drink_coke  avengers  drink_tea    q4  hot_sauce_medium  ninja  home  \
0           1         0          0   5.0                 0      0     0   
1           1         0          0   5.0                 0      0     0   
2           1         0          0  10.0                 1      0     0   
3           1         0          0   3.0                 0      0     0   
4           1         0          0   5.0                 0      0     0   

   drink_water  teenage  mutant  ...  aladdin  tokyo  monsters  name  kill  \
0            0        0       0  ...        0      0         0     0     0   
1            0        0       0  ...        0      0         0     0     0   
2            0        0       0  ...        0      0         0     0     0   
3            0        0       0  ...        0      0         0     0     0   
4            0        0       0  ...        0      0         0     0     0   

   japanese  bill  drift  hot_sauce_mild  label  
0         0     0      0      

final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup final data cleanup

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import Counter

def process_q2(text):
    if pd.isnull(text):
        return None
    numbers = re.findall(r'\b\d+(?:\.\d+)?\b', text)
    numbers = [float(num) for num in numbers]
    filtered_numbers = [num for num in numbers if 1 <= num <= 10]
    if filtered_numbers:
        result = sum(filtered_numbers) / len(filtered_numbers)
    elif ',' in text:
        return text.count(',') + 1
    else:
        result = len(text) / 5
    return np.ceil(result)

def extract_numeric(value):
    if isinstance(value, str):
        value = value.lower().strip()
        numbers = re.findall(r'\d+', value)
        if numbers:
            return int(numbers[0])
        return "unknown"
    return value



def clean_drink(value):
    drink_map = {
        "coke": ["coke", "cola", "diet coke", "pepsi", "diet pepsi", "soda", "pop", "soft", "fanta", "sprite", "ginger"],
        "juice": ["juice", "nestea", "lemonade"],
        "tea": ["tea", "matcha"],
        "milk": ["ayran", "milk", "dairy"],
        "beer": ["beer", "wine", "alcoholic", "sake", "soju"],
        "water": ["water"],
        "soup": ["miso", "soup"]
    }
    if isinstance(value, str):
        value = re.sub(r'[^a-z0-9 ]', '', value.lower().strip())
        for category, keywords in drink_map.items():
            if any(keyword in value for keyword in keywords):
                return category
    return "unknown"

def one_hot_encode(df, column_name, categories):
    if column_name in df.columns:
        df[column_name] = df[column_name].str.lower().str.strip()
        df[column_name] = df[column_name].apply(lambda x: x.split(",") if isinstance(x, str) else [])
        for category in categories:
            df[f"{column_name}_{category}"] = df[column_name].apply(lambda x: 1 if category in x else 0)
        df.drop(columns=[column_name], inplace=True)
    return df

def clean_and_process_csv(file_path):
    df = pd.read_csv(file_path, dtype=str)
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    df = df.drop_duplicates().fillna("unknown")

    df.rename(columns={
    'q1:_from_a_scale_1_to_5,_how_complex_is_it_to_make_this_food?_(where_1_is_the_most_simple,_and_5_is_the_most_complex)': 'q1',
    'q7:_when_you_think_about_this_food_item,_who_does_it_remind_you_of?':'q7',
    "q3:_in_what_setting_would_you_expect_this_food_to_be_served?_please_check_all_that_apply":'q3'
    }, inplace=True)

    # Process Q2
    q2_col = "q2:_how_many_ingredients_would_you_expect_this_food_item_to_contain?"
    if q2_col in df.columns:
        df['q2'] = df[q2_col].apply(process_q2)
        df.drop(columns=[q2_col], inplace=True)

    # Process Q4
    q4_col = "q4:_how_much_would_you_expect_to_pay_for_one_serving_of_this_food_item?"
    if q4_col in df.columns:
        df['q4'] = df[q4_col].apply(extract_numeric)
        df.drop(columns=[q4_col], inplace=True)

    # Process Q6
    q6_col = "q6:_what_drink_would_you_pair_with_this_food_item?"
    if q6_col in df.columns:
        df['q6'] = df[q6_col].apply(clean_drink)
        for cat in ["coke", "juice", "tea", "milk", "beer", "water", "soup"]:
            df[f'drink_{cat}'] = df['q6'].apply(lambda x: 1 if x == cat else 0)
        df.drop(columns=[q6_col, "q6"], inplace=True)

    # Process Q8
    q8_col = "q8:_how_much_hot_sauce_would_you_add_to_this_food_item?"

    def map_hot_sauce_response(text):
        text = text.lower().strip()
        if text == "unknown":
            return "none"
        elif text == "a little (mild)":
            return "mild"
        elif text == "a moderate amount (medium)":
            return "medium"
        elif text == "a lot (hot)":
            return "hot"
        elif text == "i will have some of this food item with my hot sauce":
            return "with_hot_sauce"
        else:
            return "unknown"

    print(df[q8_col].unique()[:10])  # show sample raw Q8 values


    if q8_col in df.columns:
        df[q8_col] = df[q8_col].astype(str).apply(map_hot_sauce_response)
        for category in ["none", "mild", "medium", "hot", "with_hot_sauce"]:
            df[f'hot_sauce_{category}'] = df[q8_col].apply(lambda x: 1 if x == category else 0)
        df.drop(columns=[q8_col], inplace=True)  # Drop original string column

    # Process Q7 and Q3 with one-hot encoding
    df = one_hot_encode(df, "q7",
                        ["parents", "siblings", "friends", "teachers", "strangers"])
    df = one_hot_encode(df, "q3",
                        ["week day lunch", "week day dinner", "weekend lunch", "weekend dinner", "at a party", "late night snack"])

    # Process Q5 - Bag of Words
    text_column = "q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?"
    word_counts = Counter()

    stop_words = set("i me my myself we our ours you your they them their what which who this that these those am is are was were be been being have has had do does did doing a an the and but if or because as until while of at by for with about between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y".split())

    # Helper to clean special characters
    def clean_special_symbols(text):
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'[^a-z\s]', '', text)  # remove anything that's not a-z or space
        text = re.sub(r'\s+', ' ', text).strip()  # clean extra spaces
        return text

    if text_column in df.columns:
        df[text_column] = df[text_column].apply(clean_special_symbols)
        df[text_column] = df[text_column].apply(
            lambda text: [word for word in text.strip().split() if word not in stop_words]
        )
        for words in df[text_column]:
            word_counts.update(words)

        unique_words = sorted(word_counts.keys())
        bow_df = pd.DataFrame(0, index=df.index, columns=unique_words)
        for idx, words in enumerate(df[text_column]):
            for word in words:
                bow_df.at[idx, word] = 1
        df.drop(columns=[text_column], inplace=True)
        df = pd.concat([df, bow_df], axis=1)


    """
        # Process Q5 - Bag of Words
    text_column = "q5:_what_movie_do_you_think_of_when_thinking_of_this_food_item?"
    word_counts = Counter()

    stop_words = set("i me my myself we our ours you your they them their what which who this that these those am is are was were be been being have has had do does did doing a an the and but if or because as until while of at by for with about between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y".split())
    if text_column in df.columns:
        df[text_column] = df[text_column].apply(lambda text: [word for word in text.lower().strip().split() if word not in stop_words] if isinstance(text, str) else [])
        for words in df[text_column]:
            word_counts.update(words)
        unique_words = sorted(word_counts.keys())
        bow_df = pd.DataFrame(0, index=df.index, columns=unique_words)
        for idx, words in enumerate(df[text_column]):
            for word in words:
                bow_df.at[idx, word] = 1
        df.drop(columns=[text_column], inplace=True)
        df = pd.concat([df, bow_df], axis=1)
    """


    # Replace non-numeric values with 0 except for 'label'
    for column in df.columns:
        if column != "label":
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0)
    print(df.columns)
    # Drop unnecessary columns
    df.drop(columns=['id'], errors='ignore', inplace=True)


    output_path = "cleaned_" + file_path
    df.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")
    return output_path